In [1]:
import pandas as pd
import numpy as np 
#print(np.__version__,pd.__version__)  # check versions

In [2]:
df = pd.read_excel("tool_simple.xlsm",sheetname="main",skiprows=2)
df.rename(columns={'Unnamed: 0': 'VALUE_TIME','production[KWh]':'realized'}, inplace=True)
df.set_index('VALUE_TIME',inplace = True)
d = df[['realized']]
#d1 =d['2016-1':'2016-8']

In [3]:
def prev(df):
    return df[:24].append(df.shift(1,'D'))

In [4]:
def prev2(df):
    x = prev(df)
    return (x + prev(x) ) / 2

In [5]:
def prev3(df):
    x = prev(df)
    y = prev(x)
    return (x + y + prev(y) ) / 3

In [6]:
def month_hour_avg(df):
    """ df transformed to monthly hourly averages"""
    b = df.copy()
    b['month'] = b.index.month
    b['hour']  = b.index.hour
    out = b.merge( b.groupby(['month','hour']).mean(),
                   how='left', left_on=['month', 'hour'], 
                   right_index=True, suffixes=('ignore',''))[df.columns]
    return out

In [7]:
def detect_suspicious_hours(d,threshold = 0.05, avg_threshold = 200):
    """detects hours with suspicious low production
           hourly value is smaller than threshold * monthly average for that hour
           monthly average for an hour is higher than an absolute threshold
    """
    out = pd.concat([d,month_hour_avg(d)],axis=1,keys=['original', 'avg_value'])
    out.columns = out.columns.droplevel(1)
    return out.query('original < avg_value * @threshold and avg_value > @avg_threshold' )
#detect_suspicious_hours(d)[:3]

In [8]:
def clear_suspicious_hours(df):
    out = df.copy()
    out.loc[detect_suspicious_hours(df).index, df.columns] = np.nan
    return out
#clear_suspicious_hours(d)[:3]

In [9]:
def replace_suspicious_values(d, compare=False):
    v = clear_suspicious_hours(d)
    x = v.copy()
    x[x.isnull()] = month_hour_avg(v)
    out = pd.concat([d,v,x],axis=1,keys=['original_production','valid_production', 'filled_production',])
    out.columns = out.columns.droplevel(1)
    return out if compare else out[['filled_production']]
#replace_suspicious_values(d, compare=True).to_clipboard()       

In [10]:
# compare raw production and statistical forecasts
def compare_forecasts(d):
    c = replace_suspicious_values(d)
    out = pd.concat([d,c,prev(d),prev2(d),prev3(d),
                     month_hour_avg(d),month_hour_avg(c)], 
                    keys=['original_production','corrected_production','prevday','prev2days','prev3days',
                          'monthavg_nocorrection','monthavg_withcorrection'], axis=1)
    out.columns = out.columns.droplevel(1)
    return out
compare_forecasts(d).to_clipboard()

In [12]:
#z = pd.DataFrame(pd.date_range(start='2016-01-01',end='2016-09-01',freq='H')[:-1], columns=['VALUE_TIME'])

#d.groupby(d.index.month).sum() / d.sum()
#d.groupby([d.index.month, d.index.hour]).mean().unstack()